This notebook is designed to replicate the capabilities of ChatGPT-4, utilizing the Gradio interface and OpenAI's API as an alternative to the standard ChatGPT website. The implementation within this notebook operates as anticipated.

In [8]:
# Importing necessary modules
import os
from dotenv import load_dotenv
import openai
import gradio as gr  # 3.32

In [9]:
# Load the environment variables from the local .env file
# NOTE: while using Jupyter Notebooks with VS Code, even when the .env
# is located in the root directory of the project, you must use ../.env
# instead of .env
load_dotenv(".env")

# Retrieving API keys from the environment
openai.api_key = os.getenv("OPENAI_API_KEY")
username = os.getenv("PII_SAFE_CHAT_USERNAME")
password = os.getenv("PII_SAFE_CHAT_PASSWORD")

In [10]:
def clear_history():
    """
    Clear the global variable `message_history`.

    This function resets the `message_history` list to its default
    state, which includes two messages: one from the user and one from
    the assistant.

    Notes
    -----
    This function directly modifies the global variable
    `message_history`.

    Examples
    --------
    >>> message_history
    [{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hello!'}]
    >>> clear_history()
    >>> message_history
    [{'role': 'user', 'content': 'You are an assistant.'}, {'role': 'assistant', 'content': 'OK'}]
    """
    # Declare that we're using the global variable `message_history`
    global message_history
    # Reset `message_history` to its default state
    message_history = [
        {"role": "user", "content": f"You are a kind helpful assistant"},
        {"role": "assistant", "content": f"OK"},
    ]

In [11]:
# Initialize the `message_history` global variable to its default state
message_history = [
    {"role": "user", "content": f"You are a kind helpful assistant"},
    {"role": "assistant", "content": f"OK"},
]

In [12]:
def predict(input):
    """
    Given a user's input, this function predicts a response by sending
    the anonymized input to the OpenAI API. This prediction is then
    appended to the message history. Finally, the function returns a
    list of pairs of consecutive messages from the history, skipping the
    pre-prompt.

    Parameters
    ----------
    input : str
        User's original input message

    Returns
    -------
    list
        A list of tuples. Each tuple contains a pair of consecutive
        messages from the message history.
    """

    # Anonymize the user input
    input_anonymized = input

    # Add the anonymized user input to the message history
    message_history.append(
        {
            "role": "user",
            "content": (f"{input_anonymized}"),
        }
    )

    # Get a completion from OpenAI's API based on the message history
    completion = openai.ChatCompletion.create(
        model="gpt-4", messages=message_history
    )

    # Extract the content of the reply from the API response
    reply_content = completion.choices[0].message.content

    # Add the reply content to the message history
    message_history.append(
        {"role": "assistant", "content": f"{reply_content}"}
    )

    # Generate a response by pairing consecutive messages in the
    # history, skipping the pre-prompt
    response = [
        (message_history[i]["content"], message_history[i + 1]["content"])
        for i in range(2, len(message_history) - 1, 2)
    ]

    # Return the response
    return response


# Start creating a Gradio Blocks interface with the 'darkhuggingface'
# theme
with gr.Blocks() as demo:
    # Add a markdown element to the interface with introductory
    # information
    gr.Markdown(
        """
    ## Welcome!
    """
    )

    # Create a chatbot component
    chatbot = gr.Chatbot()

    # Create a row layout for the textbox and the clear button
    with gr.Row():
        # Create a textbox for user input
        txt = gr.Textbox(
            show_label=False, placeholder="Enter text and press enter"
        ).style(container=False)

    # Create a button to clear the chat
    clear = gr.Button("New chat")

    # Add a submit event to the textbox that triggers the predict
    # function
    # The output is displayed in the chatbot component
    txt.submit(predict, txt, chatbot)

    # Add a submit event to the textbox that clears the textbox when
    # enter is pressed
    txt.submit(None, None, txt, _js="() => {''}")

    # Add a click event to the clear button that triggers the
    # clear_history function
    # The output is displayed in the chatbot component
    clear.click(clear_history, None, chatbot, queue=False)


In [13]:
# Launch the Gradio interface
demo.launch(
    share=True,
)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://aa2732829117f79e10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
